## 將 Transformer 和 斷詞的套件下載

In [1]:
!pip install transformers datasets evaluate rouge_score py7zr

!pip install sentencepiece
!pip install transformers
!pip install rich[jupyter]

  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.9/138.9 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.7/413.7 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 2.3 MB/s eta 0:00:00


## 資料分割



In [3]:
from datasets import load_dataset
meetingbank = load_dataset("huuuyeah/meetingbank")

train_data = meetingbank['train']
test_data = meetingbank['test']
val_data = meetingbank['validation']

def generator(data_split):
  for instance in data_split:
    instance['id'], instance['summary'], instance['transcript']

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/3.32k [00:00<?, ?B/s]

train.json:   0%|          | 0.00/88.4M [00:00<?, ?B/s]

validation.json:   0%|          | 0.00/13.2M [00:00<?, ?B/s]

test.json:   0%|          | 0.00/13.4M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5169 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/861 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/862 [00:00<?, ? examples/s]

## Preview 資料內容

In [4]:
import pandas as pd
df_train = pd.DataFrame()
df_train["transcript"] = train_data['transcript']
df_train["summary"] = train_data['summary']

In [5]:
df_train.head()

,transcript,summary
0,Please refrain from profane or obscene speech....,AS AMENDED a bill for an ordinance amending th...
1,An assessment has called out council bill 161 ...,AS AMENDED a bill for an ordinance amending th...
2,I Please close the voting. Announce the result...,AS AMENDED a bill for an ordinance amending th...
3,"Motion passes. Hey, thank you very much. Now w...",Recommendation to respectfully request City Co...
4,All right. Pursuant to Council Bill 3.7 consid...,AS AMENDED a bill for an ordinance amending th...


## 將 target + 前綴 summarize:

In [6]:

df_train['summary'] = "summarize: "+df_train['summary']
df_train.head()

,transcript,summary
0,Please refrain from profane or obscene speech....,summarize: AS AMENDED a bill for an ordinance ...
1,An assessment has called out council bill 161 ...,summarize: AS AMENDED a bill for an ordinance ...
2,I Please close the voting. Announce the result...,summarize: AS AMENDED a bill for an ordinance ...
3,"Motion passes. Hey, thank you very much. Now w...",summarize: Recommendation to respectfully requ...
4,All right. Pursuant to Council Bill 3.7 consid...,summarize: AS AMENDED a bill for an ordinance ...


In [7]:
df_train.shape

(5169, 2)

## T5 Model 訓練設定

In [8]:
# Importing libraries
import os
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
import os

# Importing the T5 modules from huggingface/transformers
from transformers import T5Tokenizer, T5ForConditionalGeneration

from rich.table import Column, Table
from rich import box
from rich.console import Console

# define a rich console logger
console=Console(record=True)

def display_df(df):
  """display dataframe in ASCII format"""

  console=Console()
  table = Table(Column("source_text", justify="center" ), Column("target_text", justify="center"), title="Sample Data",pad_edge=False, box=box.ASCII)

  for i, row in enumerate(df.values.tolist()):
    table.add_row(row[0], row[1])

  console.print(table)

training_logger = Table(Column("Epoch", justify="center" ),
                        Column("Steps", justify="center"),
                        Column("Loss", justify="center"),
                        title="Training Status",pad_edge=False, box=box.ASCII)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [9]:
# Setting up the device for GPU usage
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

## Torch Dataset 定義格式

In [10]:
class MeetingBankDataset(Dataset):
  """
  Creating a custom dataset for reading the dataset and
  loading it into the dataloader to pass it to the neural network for finetuning the model

  """

  def __init__(self, dataframe, tokenizer, source_len, target_len, source_text, target_text):
    self.tokenizer = tokenizer
    self.data = dataframe
    self.source_len = source_len
    self.summ_len = target_len
    self.target_text = self.data[target_text]
    self.source_text = self.data[source_text]

  def __len__(self):
    return len(self.target_text)

  def __getitem__(self, index):
    source_text = str(self.source_text[index])
    target_text = str(self.target_text[index])

    #cleaning data so as to ensure data is in string type
    source_text = ' '.join(source_text.split())
    target_text = ' '.join(target_text.split())

    source = self.tokenizer.batch_encode_plus([source_text], max_length= self.source_len, pad_to_max_length=True, truncation=True, padding="max_length", return_tensors='pt')
    target = self.tokenizer.batch_encode_plus([target_text], max_length= self.summ_len, pad_to_max_length=True, truncation=True, padding="max_length", return_tensors='pt')

    source_ids = source['input_ids'].squeeze()
    source_mask = source['attention_mask'].squeeze()
    target_ids = target['input_ids'].squeeze()
    target_mask = target['attention_mask'].squeeze()

    return {
        'source_ids': source_ids.to(dtype=torch.long),
        'source_mask': source_mask.to(dtype=torch.long),
        'target_ids': target_ids.to(dtype=torch.long),
        'target_ids_y': target_ids.to(dtype=torch.long)
    }

## Model Training Setting

In [11]:
def train(epoch, tokenizer, model, device, loader, optimizer):

  """
  Function to be called for training with the parameters passed from main function

  """

  model.train()
  for _,data in enumerate(loader, 0):
    y = data['target_ids'].to(device, dtype = torch.long)
    y_ids = y[:, :-1].contiguous()
    lm_labels = y[:, 1:].clone().detach()
    lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
    ids = data['source_ids'].to(device, dtype = torch.long)
    mask = data['source_mask'].to(device, dtype = torch.long)

    outputs = model(input_ids = ids, attention_mask = mask, decoder_input_ids=y_ids, labels=lm_labels)
    loss = outputs[0]

    if _%10==0:
      training_logger.add_row(str(epoch), str(_), str(loss))
      console.print(training_logger)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


## Model Validation

In [12]:
def validate(epoch, tokenizer, model, device, loader):

  """
  Function to evaluate model for predictions

  """
  model.eval()
  predictions = []
  actuals = []
  with torch.no_grad():
      for _, data in enumerate(loader, 0):
          y = data['target_ids'].to(device, dtype = torch.long)
          ids = data['source_ids'].to(device, dtype = torch.long)
          mask = data['source_mask'].to(device, dtype = torch.long)

          generated_ids = model.generate(
              input_ids = ids,
              attention_mask = mask,
              max_length=150,
              num_beams=2,
              repetition_penalty=2.5,
              length_penalty=1.0,
              early_stopping=True
              )
          preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
          target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True)for t in y]
          if _%10==0:
              console.print(f'Completed {_}')

          predictions.extend(preds)
          actuals.extend(target)
  return predictions, actuals


In [13]:
def T5Trainer(dataframe, source_text, target_text, model_params, output_dir="./outputs/" ):

  """
  T5 trainer

  """

  # Set random seeds and deterministic pytorch for reproducibility
  torch.manual_seed(model_params["SEED"]) # pytorch random seed
  np.random.seed(model_params["SEED"]) # numpy random seed
  torch.backends.cudnn.deterministic = True

  # logging
  console.log(f"""[Model]: Loading {model_params["MODEL"]}...\n""")

  # tokenzier for encoding the text
  tokenizer = T5Tokenizer.from_pretrained(model_params["MODEL"])

  # Defining the model. We are using t5-base model and added a Language model layer on top for generation of Summary.
  # Further this model is sent to device (GPU/TPU) for using the hardware.
  model = T5ForConditionalGeneration.from_pretrained(model_params["MODEL"])
  model = model.to(device)

  # logging
  console.log(f"[Data]: Reading data...\n")

  # Importing the raw dataset
  dataframe = dataframe[[source_text,target_text]]
  display_df(dataframe.head(2))


  # Creation of Dataset and Dataloader
  # Defining the train size. So 80% of the data will be used for training and the rest for validation.
  train_size = 0.8
  train_dataset=dataframe.sample(frac=train_size,random_state = model_params["SEED"])
  val_dataset=dataframe.drop(train_dataset.index).reset_index(drop=True)
  train_dataset = train_dataset.reset_index(drop=True)

  console.print(f"FULL Dataset: {dataframe.shape}")
  console.print(f"TRAIN Dataset: {train_dataset.shape}")
  console.print(f"TEST Dataset: {val_dataset.shape}\n")


  # Creating the Training and Validation dataset for further creation of Dataloader
  training_set = MeetingBankDataset(train_dataset, tokenizer, model_params["MAX_SOURCE_TEXT_LENGTH"], model_params["MAX_TARGET_TEXT_LENGTH"], source_text, target_text)
  val_set = MeetingBankDataset(val_dataset, tokenizer, model_params["MAX_SOURCE_TEXT_LENGTH"], model_params["MAX_TARGET_TEXT_LENGTH"], source_text, target_text)


  # Defining the parameters for creation of dataloaders
  train_params = {
      'batch_size': model_params["TRAIN_BATCH_SIZE"],
      'shuffle': True,
      'num_workers': 0
      }


  val_params = {
      'batch_size': model_params["VALID_BATCH_SIZE"],
      'shuffle': False,
      'num_workers': 0
      }


  # Creation of Dataloaders for testing and validation. This will be used down for training and validation stage for the model.
  training_loader = DataLoader(training_set, **train_params)
  val_loader = DataLoader(val_set, **val_params)


  # Defining the optimizer that will be used to tune the weights of the network in the training session.
  optimizer = torch.optim.Adam(params =  model.parameters(), lr=model_params["LEARNING_RATE"])


  # Training loop
  console.log(f'[Initiating Fine Tuning]...\n')

  for epoch in range(model_params["TRAIN_EPOCHS"]):
      train(epoch, tokenizer, model, device, training_loader, optimizer)

  console.log(f"[Saving Model]...\n")
  #Saving the model after training
  path = os.path.join(output_dir, "model_files")
  model.save_pretrained(path)
  tokenizer.save_pretrained(path)


  # evaluating test dataset
  console.log(f"[Initiating Validation]...\n")
  for epoch in range(model_params["VAL_EPOCHS"]):
    predictions, actuals = validate(epoch, tokenizer, model, device, val_loader)
    final_df = pd.DataFrame({'Generated Text':predictions,'Actual Text':actuals})
    final_df.to_csv(os.path.join(output_dir,'predictions.csv'))

  console.save_text(os.path.join(output_dir,'logs.txt'))

  console.log(f"[Validation Completed.]\n")
  console.print(f"""[Model] Model saved @ {os.path.join(output_dir, "model_files")}\n""")
  console.print(f"""[Validation] Generation on Validation data saved @ {os.path.join(output_dir,'predictions.csv')}\n""")
  console.print(f"""[Logs] Logs saved @ {os.path.join(output_dir,'logs.txt')}\n""")

In [14]:
model_params={
    "MODEL":"t5-base",             # model_type: t5-base/t5-large
    "TRAIN_BATCH_SIZE":8,          # training batch size
    "VALID_BATCH_SIZE":8,          # validation batch size
    "TRAIN_EPOCHS":3,              # number of training epochs
    "VAL_EPOCHS":1,                # number of validation epochs
    "LEARNING_RATE":1e-4,          # learning rate
    "MAX_SOURCE_TEXT_LENGTH":512,  # max length of source text
    "MAX_TARGET_TEXT_LENGTH":50,   # max length of target text
    "SEED": 42                     # set seed for reproducibility

}


In [15]:
tokenizer = T5Tokenizer.from_pretrained(model_params["MODEL"])
MeetingBankDataset_train = MeetingBankDataset(df_train, tokenizer, model_params["MAX_SOURCE_TEXT_LENGTH"], model_params["MAX_TARGET_TEXT_LENGTH"], source_text="transcript", target_text="summary")

# Assuming MeetingBankDataset_train is an instance of your dataset class
# and you want to inspect the first few samples
num_samples_to_inspect = 5

for i in range(num_samples_to_inspect):
    sample = MeetingBankDataset_train[i]
    print(f"Sample {i + 1}:")
    print("Source Text (transcript):", sample['source_ids'])
    print("Target Text (summary):", sample['target_ids'])
    print("=" * 50)


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Sample 1:
Source Text (transcript): tensor([  863, 27726,    45,  7108,   152,    15,    42,     3,    32,   115,
        22886,  5023,     5,  7143,    39,  2622,    12,  6098,    38,     3,
            9,   829,    11, 27726,    45,   928,    42,   525,  6032,     5,
         2063, 15893, 12798,  3706,     6,    56,    25,   754,   474,  2063,
         3259,   898,   536,    30,     8,  1501,    58,  2163,     6,  1661,
         9083,     7,     6,    27,   888,    24,  6098,  2876,   898,   536,
           38, 21012,     6,    36,  2681,  1286,   804,  4587,    11,   103,
         1903,     5,    94,    65,   118,  2301,    11,   511,    15,    26,
            5,  2063,   348,     5, 26744,     6,   281,  2177,    11,   462,
           39,  4644,    12,   856, 14545,     5,  1562,    25,     6,  1363,
            5,  1661,     5,    27,   888,    24,  6098,  2876,   898,   536,
           38, 21012,    36,   856, 21012,    16,     8,   826,  1090,     7,
           30,   543,   874,

In [16]:
len(df_train["transcript"][0])

152314

In [17]:
df_train["transcript"][0]

"Please refrain from profane or obscene speech. Direct your comments to council as a whole and refrain from individual or personal attacks. Councilwoman Gilmore, will you please put Council Bill 161 on the floor? Yes, President Brooks, I move that council bill 161 as amended, be placed upon final consideration and do pass. It has been moved and seconded. Councilman. Clerk, go ahead and offer your motion to further amend. Thank you, Mr. President. I move that council bill 161 as amended be further amended in the following particulars on page five, line one, strike 13 .1.9 and replace with 13 .1. ten on page five. Line three strike 13 1.9.3 and replace with 13 one dot and four. Add a new section to the bill which reads as follows Section six A except as otherwise provided in Section six B of this ordinance. With respect to certain site development plan applications, the amendments to the Denver Zoning Code adopted by this ordinance take effect on May five, 2017. B Notwithstanding Section

In [18]:
# Assuming MeetingBankDataset_train is an instance of your dataset class
# and you want to inspect the keys of the first sample
sample = MeetingBankDataset_train[0]

# Print the keys of the sample dictionary
print("Keys of the Sample Dictionary:", sample.keys())

# Alternatively, you can print the entire dictionary for better understanding
print("Sample Dictionary:", sample)

Keys of the Sample Dictionary: dict_keys(['source_ids', 'source_mask', 'target_ids', 'target_ids_y'])
Sample Dictionary: {'source_ids': tensor([  863, 27726,    45,  7108,   152,    15,    42,     3,    32,   115,
        22886,  5023,     5,  7143,    39,  2622,    12,  6098,    38,     3,
            9,   829,    11, 27726,    45,   928,    42,   525,  6032,     5,
         2063, 15893, 12798,  3706,     6,    56,    25,   754,   474,  2063,
         3259,   898,   536,    30,     8,  1501,    58,  2163,     6,  1661,
         9083,     7,     6,    27,   888,    24,  6098,  2876,   898,   536,
           38, 21012,     6,    36,  2681,  1286,   804,  4587,    11,   103,
         1903,     5,    94,    65,   118,  2301,    11,   511,    15,    26,
            5,  2063,   348,     5, 26744,     6,   281,  2177,    11,   462,
           39,  4644,    12,   856, 14545,     5,  1562,    25,     6,  1363,
            5,  1661,     5,    27,   888,    24,  6098,  2876,   898,   536,
      

## Summarize Result & Training Process (Loss)

In [19]:

T5Trainer(dataframe=df_train, source_text="transcript", target_text="summary", model_params=model_params, output_dir="outputs")

[17:04:07] [Model]: Loading t5-base...                                           <ipython-input-13-046fd2a512ef>:14
                                                                                                                   

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

[17:04:18] [Data]: Reading data...                                               <ipython-input-13-046fd2a512ef>:25
                                                                                                                   

                                                    Sample Data                                                    
+-----------------------------------------------------------------------------------------------------------------+
|                      source_text                       |                       target_text                      |
|--------------------------------------------------------+--------------------------------------------------------|
| Please refrain from profane or obscene speech. Direct  | summarize: AS AMENDED a bill for an ordinance amending |
| your comments to council as a whole and refrain from   | the Denver Zoning Code to revise parking exemptions for|
| individual or personal attacks. Councilwoman Gilmore,  | pre-existing small zone lots. Approves a text amendment|
|will you please put Council Bill 161 on the floor? Yes, |  to the Denver Zoning Code to revise the Pre-Existing  |
|   President Brooks, I move that council bill 161 as    |     Small Zone Lot parking exemption. The Committee    |
|  amended, be placed upon final consideration and do    | approved filing this bill at its meeting on 2-14-17. On|
|   pass. It has been moved and seconded. Councilman.    |     2-27-17, Council held this item in Committee to    |
|Clerk, go ahead and offer your motion to further amend. |   3-20-17. Amended 3-20-17 to ensure that the parking  |
|Thank you, Mr. President. I move that council bill 161  |     exemption is applied for all uses. Some parking    |
|    as amended be further amended in the following      |  requirements are calculated based on gross floor area |
|particulars on page five, line one, strike 13 .1.9 and  | while others are on number of units and not explicitly |
|replace with 13 .1. ten on page five. Line three strike |      for gross floor area, to further clarify the      |
| 13 1.9.3 and replace with 13 one dot and four. Add a   |  legislative intent of the proposed bill to emphasize  |
|new section to the bill which reads as follows Section  |  the city’s commitment to more comprehensively address |
|six A except as otherwise provided in Section six B of  |   transportation demand management strategies in the   |
|     this ordinance. With respect to certain site       |     short term, and to require a Zoning Permit with    |
| development plan applications, the amendments to the   |      Informational Notice for all new buildings on     |
|   Denver Zoning Code adopted by this ordinance take    |  Pre-Existing Small Zone Lots that request to use the  |
|effect on May five, 2017. B Notwithstanding Section six | small lot parking exemption; Enables all expansions to |
| A of this ordinance, if requested by an applicant, a   |     existing buildings to receive the full parking     |
|pending formal site development plan application may be |   exemption; and clarifies at what point an “existing  |
|  processed under the provisions of the Denver Zoning   |                 building” is considered                |
|Code concerning the small LA parking exemption prior to |                                                        |
| the adoption of this ordinance. The prior small loft   |                                                        |
|     parking exemption. If CPD receives a complete      |                                                        |
|application for a mandatory concept review pursuant to  |                                                        |
|section 12 .3.2.2 of the Denver Zoning Code, which was  |                                                        |
|  submitted in advance of a required site development   |                                                        |
|  plan per section 12 .4.3.3 on or before August 26,    |                                                        |
|  2016, and such application sought to use the prior    |                                                        |
|small loft parking exemption . Great. It has been moved

...

                               Training Status                               
+---------------------------------------------------------------------------+
|Epoch | Steps |                            Loss                            |
|------+-------+------------------------------------------------------------|
|  0   |   0   | tensor(7.2021, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  10   | tensor(3.6671, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  20   | tensor(2.8000, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  30   | tensor(2.2624, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  40   | tensor(3.3401, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  50   | tensor(1.8915, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  60   | tensor(2.3670, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  70   | tensor(1.9828, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  80   | tensor(1.8418, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  90   | tensor(1.9107, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  100  | tensor(1.7094, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  110  | tensor(2.1755, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  120  | tensor(2.2644, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  130  | tensor(1.5328, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  140  | tensor(2.1764, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  150  | tensor(1.4268, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  160  | tensor(1.6227, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  170  | tensor(2.2153, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  180  | tensor(1.4890, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  190  | tensor(1.4742, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  200  | tensor(3.1447, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  210  | tensor(1.7869, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  220  | tensor(1.9425, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  230  | tensor(0.7442, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  240  | tensor(1.9097, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  250  | tensor(1.3800, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  260  | tensor(1.4863, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  270  | tensor(1.4175, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  280  | tensor(2.2509, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  290  | tensor(1.5530, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  300  | tensor(1.7723, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  310  | tensor(2.2952, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  320  | tensor(1.4672, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  330  | tensor(1.8552, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  340  | tensor(1.3404, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  350  | tensor(1.8787, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  360  | tensor(2.1364, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  370  | tensor(1.7656, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  380  | tensor(1.4483, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  390  | tensor(1.3408, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  400  | tensor(1.2900, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  410  | tensor(1.8561, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  420  | tensor(1.9697, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  430  | tensor(1.5808, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  440  | tensor(1.7947, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  450  | tensor(1.0431, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  460  | tensor(1.5552, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  470  | tenso

                               Training Status                               
+---------------------------------------------------------------------------+
|Epoch | Steps |                            Loss                            |
|------+-------+------------------------------------------------------------|
|  0   |   0   | tensor(7.2021, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  10   | tensor(3.6671, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  20   | tensor(2.8000, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  30   | tensor(2.2624, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  40   | tensor(3.3401, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  50   | tensor(1.8915, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  60   | tensor(2.3670, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  70   | tensor(1.9828, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  80   | tensor(1.8418, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  90   | tensor(1.9107, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  100  | tensor(1.7094, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  110  | tensor(2.1755, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  120  | tensor(2.2644, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  130  | tensor(1.5328, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  140  | tensor(2.1764, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  150  | tensor(1.4268, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  160  | tensor(1.6227, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  170  | tensor(2.2153, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  180  | tensor(1.4890, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  190  | tensor(1.4742, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  200  | tensor(3.1447, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  210  | tensor(1.7869, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  220  | tensor(1.9425, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  230  | tensor(0.7442, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  240  | tensor(1.9097, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  250  | tensor(1.3800, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  260  | tensor(1.4863, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  270  | tensor(1.4175, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  280  | tensor(2.2509, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  290  | tensor(1.5530, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  300  | tensor(1.7723, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  310  | tensor(2.2952, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  320  | tensor(1.4672, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  330  | tensor(1.8552, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  340  | tensor(1.3404, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  350  | tensor(1.8787, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  360  | tensor(2.1364, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  370  | tensor(1.7656, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  380  | tensor(1.4483, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  390  | tensor(1.3408, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  400  | tensor(1.2900, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  410  | tensor(1.8561, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  420  | tensor(1.9697, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  430  | tensor(1.5808, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  440  | tensor(1.7947, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  450  | tensor(1.0431, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  460  | tensor(1.5552, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  470  | tenso

                               Training Status                               
+---------------------------------------------------------------------------+
|Epoch | Steps |                            Loss                            |
|------+-------+------------------------------------------------------------|
|  0   |   0   | tensor(7.2021, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  10   | tensor(3.6671, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  20   | tensor(2.8000, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  30   | tensor(2.2624, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  40   | tensor(3.3401, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  50   | tensor(1.8915, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  60   | tensor(2.3670, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  70   | tensor(1.9828, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  80   | tensor(1.8418, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  90   | tensor(1.9107, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  100  | tensor(1.7094, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  110  | tensor(2.1755, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  120  | tensor(2.2644, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  130  | tensor(1.5328, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  140  | tensor(2.1764, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  150  | tensor(1.4268, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  160  | tensor(1.6227, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  170  | tensor(2.2153, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  180  | tensor(1.4890, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  190  | tensor(1.4742, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  200  | tensor(3.1447, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  210  | tensor(1.7869, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  220  | tensor(1.9425, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  230  | tensor(0.7442, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  240  | tensor(1.9097, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  250  | tensor(1.3800, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  260  | tensor(1.4863, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  270  | tensor(1.4175, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  280  | tensor(2.2509, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  290  | tensor(1.5530, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  300  | tensor(1.7723, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  310  | tensor(2.2952, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  320  | tensor(1.4672, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  330  | tensor(1.8552, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  340  | tensor(1.3404, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  350  | tensor(1.8787, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  360  | tensor(2.1364, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  370  | tensor(1.7656, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  380  | tensor(1.4483, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  390  | tensor(1.3408, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  400  | tensor(1.2900, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  410  | tensor(1.8561, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  420  | tensor(1.9697, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  430  | tensor(1.5808, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  440  | tensor(1.7947, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  450  | tensor(1.0431, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  460  | tensor(1.5552, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  470  | tenso

                               Training Status                               
+---------------------------------------------------------------------------+
|Epoch | Steps |                            Loss                            |
|------+-------+------------------------------------------------------------|
|  0   |   0   | tensor(7.2021, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  10   | tensor(3.6671, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  20   | tensor(2.8000, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  30   | tensor(2.2624, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  40   | tensor(3.3401, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  50   | tensor(1.8915, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  60   | tensor(2.3670, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  70   | tensor(1.9828, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  80   | tensor(1.8418, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  90   | tensor(1.9107, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  100  | tensor(1.7094, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  110  | tensor(2.1755, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  120  | tensor(2.2644, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  130  | tensor(1.5328, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  140  | tensor(2.1764, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  150  | tensor(1.4268, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  160  | tensor(1.6227, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  170  | tensor(2.2153, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  180  | tensor(1.4890, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  190  | tensor(1.4742, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  200  | tensor(3.1447, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  210  | tensor(1.7869, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  220  | tensor(1.9425, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  230  | tensor(0.7442, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  240  | tensor(1.9097, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  250  | tensor(1.3800, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  260  | tensor(1.4863, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  270  | tensor(1.4175, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  280  | tensor(2.2509, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  290  | tensor(1.5530, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  300  | tensor(1.7723, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  310  | tensor(2.2952, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  320  | tensor(1.4672, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  330  | tensor(1.8552, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  340  | tensor(1.3404, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  350  | tensor(1.8787, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  360  | tensor(2.1364, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  370  | tensor(1.7656, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  380  | tensor(1.4483, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  390  | tensor(1.3408, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  400  | tensor(1.2900, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  410  | tensor(1.8561, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  420  | tensor(1.9697, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  430  | tensor(1.5808, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  440  | tensor(1.7947, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  450  | tensor(1.0431, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  460  | tensor(1.5552, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  470  | tenso

KeyboardInterrupt: 

In [ ]:
import re
import matplotlib.pyplot as plt

# Open a file in read mode
file_path = '/content/outputs/logs.txt'

try:
    with open(file_path, 'r') as file:
        # Read the entire contents of the file
        content = file.read()
        print(content)
except FileNotFoundError:
    print(f"The file at '{file_path}' was not found.")
except Exception as e:
    print(f"An error occurred: {e}")

# Define a regular expression pattern to extract the numeric values
pattern = re.compile(r'tensor\(([\d.]+)')

# Find all matches in the provided text
matches = pattern.findall(content)

# Convert the matched strings to float values
loss_values = [float(match) for match in matches]

# Print the extracted loss values
print("Extracted Loss Values:", loss_values)
print(loss_values[-5*model_params["TRAIN_EPOCHS"] : ])
final_loss = loss_values[-51*model_params["TRAIN_EPOCHS"] : ]

plt.style.use('ggplot')
plt.plot(final_loss)
plt.xlabel("Step")
plt.ylabel("Loss")
plt.title("Training Process")

The file at '/content/outputs/logs.txt' was not found.


NameError: ignored

# -----------------------------------------------------------------------------------------

In [ ]:
import pandas as pd

prediction_text = pd.read_csv("/content/outputs/predictions.csv")
prediction_text

,Unnamed: 0,Generated Text,Actual Text
0,0,referred to the Federal Legislation Committee ...,summarize: Recommendation to respectfully requ...
1,1,a bill for an ordinance amending Sections 2 an...,summarize: AS AMENDED a bill for an ordinance ...
2,2,adopted as amended. (Citywide),summarize: AS AMENDED a bill for an ordinance ...
3,3,notified the City Clerk of the Certificate of ...,summarize: Report of the City Clerk on the Cer...
4,4,a bill for an ordinance amending and revising ...,summarize: A bill for an ordinance approving S...
...,...,...,...
1029,1029,a map showing the location of the Alameda Poin...,summarize: Presentation on Status Report of En...
1030,1030,approved by relating to the Seattle Fire Depar...,summarize: AN ORDINANCE relating to the Seattl...
1031,1031,requesting King County and the State of Washin...,summarize: A RESOLUTION requesting King County...
1032,1032,dealt with the destruction of records for vari...,summarize: Recommendation to approve the destr...


In [ ]:
from rouge_score import rouge_scorer
import pandas as pd

# Assuming prediction_text is a Pandas DataFrame
# prediction_text["Generated Text"] = prediction_text["Generated Text"].astype(str).apply(lambda x: x.lower())
# prediction_text["Actual Text"] = prediction_text["Actual Text"].astype(str).apply(lambda x: x.lower())

# Initialize Rouge scorer
scorer = rouge_scorer.RougeScorer(['rouge1'], use_stemmer=True)

# Calculate Rouge scores for each row
scores_list = []
for index, row in prediction_text.iterrows():
    scores = scorer.score(row["Generated Text"], row["Actual Text"])
    scores_list.append(scores)

scores_df = pd.DataFrame(scores_list)

# Extract and print the F-measure
f_measure_rouge1 = scores_df['rouge1'].apply(lambda x: x.fmeasure)
precision_rouge1 = scores_df['rouge1'].apply(lambda x: x.precision)
print("F-measure for Rouge1:", f_measure_rouge1.mean())
print("Precision for Rouge1:", precision_rouge1.mean())

F-measure for Rouge1: 0.5855230399477434
Precision for Rouge1: 0.5426063353140064


In [ ]:
scores

{'rouge1': Score(precision=0.35135135135135137, recall=0.3333333333333333, fmeasure=0.34210526315789475)}

## 讀取訓練好的模型

In [ ]:
max_target_length = 50
output_dir = "./outputs/"
path = os.path.join(output_dir, "model_files")


model = T5ForConditionalGeneration.from_pretrained(path)
tokenizer = T5Tokenizer.from_pretrained(path)
model = model.to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:

import os
import numpy as np
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer
from rouge import Rouge
from tqdm.auto import tqdm



def test_loop(dataloader, model):
    preds, labels = [], []

    model.eval()
    for batch_data in tqdm(dataloader):
        batch_data = batch_data.to(device)
        with torch.no_grad():
            generated_tokens = model.generate(
                batch_data["input_ids"],
                attention_mask=batch_data["attention_mask"],
                max_length=max_target_length,
                num_beams=4,
                no_repeat_ngram_size=2,
            ).cpu().numpy()
        if isinstance(generated_tokens, tuple):
            generated_tokens = generated_tokens[0]
        label_tokens = batch_data["labels"].cpu().numpy()
        tokenizer =  T5Tokenizer.from_pretrained(path)
        decoded_preds = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
        label_tokens = np.where(label_tokens != -100, label_tokens, tokenizer.pad_token_id)
        decoded_labels = tokenizer.batch_decode(label_tokens, skip_special_tokens=True)

        preds += [' '.join(pred.strip()) for pred in decoded_preds]
        labels += [' '.join(label.strip()) for label in decoded_labels]
    scores = rouge.get_scores(hyps=preds, refs=labels, avg=True)
    result = {key: value['f'] * 100 for key, value in scores.items()}
    result['avg'] = np.mean(list(result.values()))
    print(f"Rouge1: {result['rouge-1']:>0.2f} Rouge2: {result['rouge-2']:>0.2f} RougeL: {result['rouge-l']:>0.2f}\n")
    return result

In [ ]:

val_loader = DataLoader(est_data, **val_params)
test_dataloader = DataLoader(test_data, batch_size=32, shuffle=False, collate_fn=collote_fn)

test_loop(test_dataloader, model)

NameError: ignored

In [ ]:
import pandas as pd
train_data["summary"][0]==train_data["summary"][1]
train_data["transcript"][0]==train_data["transcript"][1]


False

In [ ]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem.porter import *
import re
nltk.download("stopwords")
nltk.download('punkt')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
passage = train_data["summary"][0]

In [ ]:
contractions = {
"ain't": "am not / are not / is not / has not / have not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is / how does",
"I'd": "I had / I would",
"I'd've": "I would have",
"I'll": "I shall / I will",
"I'll've": "I shall have / I will have",
"I'm": "I am",
"I've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"
}

## 1. Word Counts：
## 這種技術基於這樣的假設，即重要的單詞在一段文本中出現的頻率更高。

- ## 參數1: Noun score : 由於並非所有的單詞都是同等重要的，所以我們首先會使用詞性標記來識別一個句子中的名詞，然後根據句子中的名詞總數給每個句子打分。
- ## 參數2 : Numeric score : 數字很重要，因為他們對任何活動可以提供更精確的信息。例如 “I ate 3 apples”。數字也可以是表示一個時間段的年份，因此為每個句子分配一個數字分數是很重要的。
- ## 參數3 : Word Counts : 每個單詞的計數越高，表示越重要。

# 類似地計算標準化Numeric score和標準化Word Counts。最後把每個句子的分數加起來，然後計算最後的分數。

In [ ]:
contractions_re = re.compile('(%s)' % '|'.join(contractions.keys()))
def expand_contractions(s, contractions_dict=contractions):
    def replace(match):
        return contractions_dict[match.group(0)]
    return contractions_re.sub(replace, s)

sentences = sent_tokenize(passage)
sentences = [expand_contractions(i) for i in sentences]
sentences = [re.sub('\n', '', i) for i in sentences]

In [ ]:
def create_freq_table(text_string):
    stopwords_list = set(stopwords.words('english'))

    words = word_tokenize(text_string)

    ps = PorterStemmer()

    freq_table = {}

    for word in words:
        #stem word
        word = ps.stem(word)

        #remove stopwords
        if word in stopwords_list:
            continue
        elif word in freq_table:
            freq_table[word] += 1
        else:
            freq_table[word] = 1

    return freq_table
freq_table = create_freq_table(" ".join(sentences))

In [ ]:
def create_freq_table(text_string):
    stopwords_list = set(stopwords.words('english'))

    words = word_tokenize(text_string)

    ps = PorterStemmer()

    freq_table = {}

    for word in words:
        #stem word
        word = ps.stem(word)

        #remove stopwords
        if word in stopwords_list:
            continue
        elif word in freq_table:
            freq_table[word] += 1
        else:
            freq_table[word] = 1

    return freq_table

freq_table = create_freq_table(" ".join(sentences))
freq_table

{'amend': 4,
 'bill': 3,
 'ordin': 1,
 'denver': 2,
 'zone': 6,
 'code': 2,
 'revis': 2,
 'park': 6,
 'exempt': 5,
 'pre-exist': 3,
 'small': 4,
 'lot': 4,
 '.': 5,
 'approv': 2,
 'text': 1,
 'committe': 2,
 'file': 1,
 'thi': 2,
 'meet': 1,
 '2-14-17': 1,
 '2-27-17': 1,
 ',': 3,
 'council': 1,
 'held': 1,
 'item': 1,
 '3-20-17': 2,
 'ensur': 1,
 'appli': 1,
 'use': 2,
 'requir': 2,
 'calcul': 1,
 'base': 1,
 'gross': 2,
 'floor': 2,
 'area': 2,
 'number': 1,
 'unit': 1,
 'explicitli': 1,
 'clarifi': 2,
 'legisl': 1,
 'intent': 1,
 'propos': 1,
 'emphas': 1,
 'citi': 1,
 '’': 1,
 'commit': 1,
 'comprehens': 1,
 'address': 1,
 'transport': 1,
 'demand': 1,
 'manag': 1,
 'strategi': 1,
 'short': 1,
 'term': 1,
 'permit': 1,
 'inform': 1,
 'notic': 1,
 'new': 1,
 'build': 3,
 'request': 1,
 ';': 2,
 'enabl': 1,
 'expans': 1,
 'exist': 2,
 'receiv': 1,
 'full': 1,
 'point': 1,
 '“': 1,
 '”': 1,
 'consid': 1}

In [ ]:
!pip install diffusers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 16.9 MB/s eta 0:00:00


In [ ]:


from diffusers import DiffusionPipeline

pipeline = DiffusionPipeline.from_pretrained("SG161222/Realistic_Vision_V5.1_noVAE")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

model_index.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

text_encoder/config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/492M [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

## Fine-tunging 基本